# RehabModel

This notebook takes all of the data outputs from the various GetData notebooks, combines them, and creates a model to predict the drug poison mortality rate.

In [1]:
%matplotlib inline

from bs4 import BeautifulSoup
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import string
import requests
import time

Pull the dictionaries from the json files, convert them to dataframes, and set the index to an integer type.

In [2]:
with open('DataRehabCenters.json', 'r') as f:
    rehab_dict = json.load(f);
with open('DataMortality.json', 'r') as f:
    mortality_dict = json.load(f);
with open('DataPrescriptionMedicare.json', 'r') as f:
    prescription_dict = json.load(f);

rehab_df = pd.DataFrame(rehab_dict);

mortality_df = pd.DataFrame(mortality_dict);
mortality_df.index = mortality_df.index.astype(int);
mortality_df.sort_index(inplace=True);

prescription_df = pd.DataFrame(prescription_dict);
prescription_df.index = prescription_df.index.astype(int);
prescription_df.sort_index(inplace=True);

Set consistent labels! Print the data types of the columns to scan for potential issues. Many of the mortality dataframe objects should be integers, but are just strings.

In [3]:
rename_dict = {'state':'ST', 'county':'County'};
rehab_df.rename(columns=rename_dict, inplace=True);

print "Rehab df types:"
print rehab_df.dtypes
print "\nMortality df types:"
print mortality_df.dtypes
print "\nPrescription df types:"
print prescription_df.dtypes

Rehab df types:
County          object
n_facilities     int64
ST              object
dtype: object

Mortality df types:
County                                                          object
Estimated Age-adjusted Death Rate, 11 Categories (in ranges)    object
FIPS                                                            object
FIPS State                                                      object
Population                                                      object
ST                                                              object
State                                                           object
Year                                                            object
dtype: object

Prescription df types:
County                      object
FIPS                         int64
FIPS State                   int64
Opioid Claims              float64
Opioid Prescribing Rate    float64
Overall Claims             float64
Part D Prescribers         float64
ST                          o

Set the FIPS, FIPS State, Population, and Year columns as integer type for the mortality dict. This requires first replacing NaNs as some value, and resetting the values to NaN after the conversion. (This actually promotes the data type from integer to float, but since the real desire is for a numerical type, this is fine.)

In [4]:
mortality_df.fillna('-1', inplace=True);

mortality_df['FIPS'] = mortality_df['FIPS'].astype(int);
mortality_df['FIPS State'] = mortality_df['FIPS State'].astype(int);
mortality_df['Population'] = mortality_df['Population'].astype(int);
mortality_df['Year'] = mortality_df['Year'].astype(int);

mortality_df.replace({'Population': {-1:np.nan}}, inplace=True);

mortality_df.head()

,County,"Estimated Age-adjusted Death Rate, 11 Categories (in ranges)",FIPS,FIPS State,Population,ST,State,Year
0,"Autauga County, AL",2.1-4,1001,1,42963,AL,Alabama,1999
1,"Autauga County, AL",4.1-6,1001,1,44021,AL,Alabama,2000
2,"Autauga County, AL",4.1-6,1001,1,44889,AL,Alabama,2001
3,"Autauga County, AL",4.1-6,1001,1,45909,AL,Alabama,2002
4,"Autauga County, AL",4.1-6,1001,1,46800,AL,Alabama,2003


Add a new column that converts "Estimated Age-adjusted Death Rate" to a numerical type. Pull a list of all possible values of the original columns by grouping the dataframe (e.g., 2.1-4 or >20), split the string on punctuation, take the last word (e.g., 4 or 20), and add one for the case of '>20', otherwise subtract 1.

In [5]:
grouped_df = mortality_df.groupby('Estimated Age-adjusted Death Rate, 11 Categories (in ranges)');

death_rate_map = {};
death_rate_list = grouped_df.groups.keys();
for rate in death_rate_list:
    if('>' in rate):
        death_rate_map[rate] = int(rate.split('>')[-1])+1;
    else:
        death_rate_map[rate] = int(rate.split('-')[-1])-1;

mortality_df['Death Rate Category Median'] = [death_rate_map[rate] for rate in
    mortality_df['Estimated Age-adjusted Death Rate, 11 Categories (in ranges)'].values];
mortality_df.head()

,County,"Estimated Age-adjusted Death Rate, 11 Categories (in ranges)",FIPS,FIPS State,Population,ST,State,Year,Death Rate Category Median
0,"Autauga County, AL",2.1-4,1001,1,42963,AL,Alabama,1999,3
1,"Autauga County, AL",4.1-6,1001,1,44021,AL,Alabama,2000,5
2,"Autauga County, AL",4.1-6,1001,1,44889,AL,Alabama,2001,5
3,"Autauga County, AL",4.1-6,1001,1,45909,AL,Alabama,2002,5
4,"Autauga County, AL",4.1-6,1001,1,46800,AL,Alabama,2003,5


In order to join the datasets, the county names need to be as good a match as possible. The common errors are when the county is listed as a parish or borough, and how the word 'saint' is written (i.e., Saint, St, or St.). This function makes everything lower case, calls everything a county, and sets all versions of 'saint' to 'saint'.

In [6]:
def fix_county(word):
    new_word = word.lower().replace('.', '');
    new_word = new_word.replace('saint','st');
    new_word = new_word.split(',')[0]
    if ('county' in new_word) and ('parish' in new_word):
        new_word = new_word.replace(' parish','');
    if ('county' in new_word) and ('borough' in new_word):
        new_word = new_word.replace(' borough','');
    if 'parish' in new_word:
        new_word = new_word.replace('parish','county');
    if 'borough' in new_word:
        new_word = new_word.replace('borough','county');
    if 'county' not in new_word:
        new_word = ' '.join([new_word, 'county']);
    return new_word

Use the above function to standardize the county names. Manually fix the outstanding issues.

In [7]:
rehab_df['County']        = rehab_df['County'].apply(fix_county);
mortality_df['County']    = mortality_df['County'].apply(fix_county);
prescription_df['County'] = prescription_df['County'].apply(fix_county);

rehab_df.set_value(183,  'County', "prince george's county");
rehab_df.set_value(299,  'County', 'la porte county');
rehab_df.set_value(667,  'County', 'kauai county');
rehab_df.set_value(1068, 'County', "anchorage county");
rehab_df.set_value(1117, 'County', "queen anne's county");
mortality_df.set_value(10288, 'County', "lasalle county");
mortality_df.set_value(36525, 'County', "mckean county");

Merge all of the datasets!

Merge the mortality and prescription dataframes on several columns that should match. There are several outstanding mismatches in the county names between these dataframes that almost entirely encompass city/county entities that are inconsistently labeled. The remaining mismatches are from the 'unorganized borough' of Alaska, which will be dropped later. Thus, the county is not used as a key, and the county column from the prescription dataframe is dropped.

The rehab dataframe is merged with the result without special considerations.

In [8]:
df = pd.merge(mortality_df, prescription_df, on=['FIPS', 'FIPS State', 'ST', 'State', 'Year'], how='outer');
df.drop('County_y', axis=1, inplace=True);
df.rename(columns={'County_x':'County'}, inplace=True);
df = pd.merge(df, rehab_df, on=['County', 'ST'], how='outer');
print df.shape;
df.head()

(50261, 14)


,County,"Estimated Age-adjusted Death Rate, 11 Categories (in ranges)",FIPS,FIPS State,Population,ST,State,Year,Death Rate Category Median,Opioid Claims,Opioid Prescribing Rate,Overall Claims,Part D Prescribers,n_facilities
0,autauga county,2.1-4,1001,1,42963,AL,Alabama,1999,3,NaN,NaN,NaN,NaN,NaN
1,autauga county,4.1-6,1001,1,44021,AL,Alabama,2000,5,NaN,NaN,NaN,NaN,NaN
2,autauga county,4.1-6,1001,1,44889,AL,Alabama,2001,5,NaN,NaN,NaN,NaN,NaN
3,autauga county,4.1-6,1001,1,45909,AL,Alabama,2002,5,NaN,NaN,NaN,NaN,NaN
4,autauga county,4.1-6,1001,1,46800,AL,Alabama,2003,5,NaN,NaN,NaN,NaN,NaN


In order to use data where all fields are present, only keep records from 2013 and 2014. Fill the missing values for the number of facilities with 0, since there were no facilities found when scraping for these counties.

Show the remaining values with missing data. These are largely from unconsistently labeled versions of Alaska's unorganized borough. The remaining minority of rows with missing data come randomly from different states. However, they represent less than 2.5% of the total number of counties, so it is not egregious to simply drop these rows. There are still 3073 of 3144 (from Wikipedia) county like entities that remain. The resultant dataframe has no missing data and is ready for modeling!

In [9]:
df = df[df['Year'] > 2012]
df['n_facilities'].fillna(0, inplace=True)
print df.shape
nullmask = pd.isnull(df)
print np.sum(nullmask)
print len(df[pd.isnull(df['Overall Claims'])].groupby('County').groups.keys())
df.dropna(inplace=True);
print df.shape
print len(df.groupby(['County','ST']).groups.keys())

(6291, 14)
County                                                           10
Estimated Age-adjusted Death Rate, 11 Categories (in ranges)     10
FIPS                                                              0
FIPS State                                                        0
Population                                                       17
ST                                                                0
State                                                             0
Year                                                              0
Death Rate Category Median                                       10
Opioid Claims                                                   144
Opioid Prescribing Rate                                         144
Overall Claims                                                  144
Part D Prescribers                                              144
n_facilities                                                      0
dtype: int64
74
(6137, 14)
3073
